## Importando do TripAdvisor com beautifulsoup
#### https://stackoverflow.com/questions/50422110/basic-python-beautifulsoup-web-scraping-tripadvisor-reviews-and-data-cleaning

In [1]:
# import libraries
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import json
import re
import sys
####
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
####
# Opções para a instância do Chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument("--incognito")
#chrome_options.add_argument("--headless")  # Retirar comentário para Browser sem janela

# Configura o Driver do Chrome, com o caminho na máquina local
driver = webdriver.Chrome(options=chrome_options, 
    executable_path="./chromedriver.exe")

# PARTE DO JSON
# criar um dicionário para colocar os elementos do JSON
dicionario = {}
# criar um array vazio de elementos do tipo 'revisoes', para o dicionario
dicionario['revisoes'] = []

# inicializar as variáveis para o arquivo JSON
idRev=pontos=titulo=textoRevisao=nomeUsuario=localUsuario=dataExperiencia=""

# Função para escrever o arquivo JSON
def writeToJSONFile(path, fileName, dicionario):
    filePathNameWExt = path + '/' + fileName + '.json'
    with open(filePathNameWExt, 'w', encoding='utf-8') as fp:
        json.dump(dicionario, fp, indent=4, ensure_ascii=False)

# Função para criar a estrutura de dados JSON que será serializada
def addDocumentToJSON(idRev,nomeUsuario,localUsuario,titulo,textoRevisao,dataExperiencia,pontos):
# Cria a estrutura JSON com os registros lidos através do CSS Selector
    dicionario['revisoes'].append({  
        'IdRevisao': idRev,
        'NomeUsuario': nomeUsuario,
        'LocalUsuario': localUsuario,
        'Titulo': titulo,
        'TextoRevisao': textoRevisao,
        'DataExperiencia': dataExperiencia,
        'Pontos': pontos
    })   
    print("IdRevisao: {}".format(idRev))
    print("NomeUsuario: {}".format(nomeUsuario))
    print("LocalUsuario: {}".format(localUsuario))
    print("Titulo: {}".format(titulo))
    print("TextoRevisao: {}".format(textoRevisao))
    print("DataExperiencia: {}".format(dataExperiencia))
    print("Pontos: {}".format(pontos))
    
def initializeVarsJSON():
    # Variáveis para o JSON
    idRev = ""
    nomeUsuario = ""
    localUsuario = ""
    titulo = ""
    textoRevisao = ""
    dataExperiencia = ""
    pontos = ""
    #print("Variáveis JSON inicializadas...")

In [2]:
# Array com as linhas da tabela de dados, de todas as páginas
initializeVarsJSON()
rows = []
urlRow = []

def click_leia_mais():    
    # elemento leia Mais: span.taLnk.ulBlueLinks
    all_more = driver.find_element(By.CSS_SELECTOR, "span.taLnk.ulBlueLinks")
    try:
        all_more.click()
        time.sleep(2)
    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
        time.sleep(2)
        print(str(e))
        return
        
# inicia o drive do Chrome
url = "https://www.tripadvisor.com.br/Attraction_Review-g303572-d8820319-Reviews-Oktoberfest_Blumenau-Blumenau_State_of_Santa_Catarina.html#REWIEWS"

def get_init_driver():

    # realiza a consulta da URL
    driver.get(url)
    time.sleep(5)
    
    # elemento leia Mais: span.taLnk.ulBlueLinks
    all_more = driver.find_element(By.CSS_SELECTOR, "span.taLnk.ulBlueLinks")
    try:
        all_more.click()
        time.sleep(2)
    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
        time.sleep(2)
        print(str(e))
        return
    
    # Retorno do Driver do Selenium em html    
    html = (driver.page_source).encode('utf-8')
    extract_reviews(html, driver)

def extract_reviews(html, driver):    
    # Recebe o retorno do Driver Selenium em formato HTML e passa para o Beautiful Soup
    bs = BeautifulSoup(html,'lxml')

    # coleta os elementos de uma revisão
    for item in bs.find_all('div', {'class':'review-container'}): 
        # Id da revisão
        idRev = item.find(class_='reviewSelector')['data-reviewid']
        # Nome do usuário
        if item.find(class_='username'):
            nomeUsuario = item.find(class_="username").get_text(strip=True) 
        else:
            u = item.find(class_='info_text')
            nomeUsuario = u.find('div').get_text()    
        # Localizacao do usuário
        if item.find(class_="userLoc"):
            localUsuario = item.find('div', {'class':'userLoc'}).strong.get_text()
            #localUsuario = loc.find('strong').get_text()
        else:
            localUsuario = "none"
        # data da experiência
        if item.find(class_='prw_rup prw_reviews_stay_date_hsx'):
            dataExperiencia = item.find('div', {'class':'prw_rup prw_reviews_stay_date_hsx'}).get_text()
        else:
            dataExperiencia = ""
        # valor da avaliação realizada pelo usuário
        var = item.find(class_="ui_bubble_rating").attrs['class']   # texto do atributo da classe
        pontos = int(var[1].split("_")[-1])/10  # retira o valor da avaliação        
        # Título da revisão
        titulo = item.find('div',{'class':'quote'}).span.get_text()
        # Texto da Revisão  
        textoRevisao = item.find(class_="partial_entry").get_text(strip=True) 

        #print(idRev,nomeUsuario,localUsuario,dataExperiencia,titulo,textoRevisao,pontos)        
        addDocumentToJSON(idRev, nomeUsuario, localUsuario, titulo, textoRevisao, dataExperiencia, pontos)
        initializeVarsJSON()
        
    print('==== Próxima Página =======')
    
      
    
    navigation = bs.find("div", {"class" : "unified ui_pagination"})
    # verifica se existe Próxima Página, senão encerra a coleta e grava o arquivo JSON
    if navigation.find("a", "nav next ui_button primary disabled") == None:
        # clicar no botão próximo
        link = driver.find_element(By.CSS_SELECTOR, "a.nav.next.taLnk.ui_button.primary")
        try:
            link.click()
            time.sleep(2)
        except (TimeoutException, NoSuchElementException) as e:
            time.sleep(2)
            print(str(e))
            return
        
        click_leia_mais()
            
        # Retorno do Driver do Selenium em html     
        html = (driver.page_source).encode('utf-8')
        # chama o método recursivamente
        extract_reviews(html, driver)
    else:
        print("===== Última Página =======")        
        driver.quit()
        # Escreve os registros lidos em um arquivo texto
        writeToJSONFile('./','tripAdvisorCrawlerData', dicionario)

sys.setrecursionlimit(1500)           

if __name__ == '__main__':
    get_init_driver()
   

IdRevisao: 714631034
NomeUsuario: Patycar19
LocalUsuario: Blumenau, SC
Titulo: Oktoberfest chegando!
TextoRevisao: Outubro é o mês da maior festa da cerveja no Brasil. A Oktoberfest atrai muitas pessoas não somente pelas cervejas mas também pela gastronomia alemã. O Parque Vila Germânica fica muito bonito, com várias atrações e opções para comer e beber. As fotos são de 2018.
DataExperiencia: Data da experiência: outubro de 2019
Pontos: 5.0
IdRevisao: 713455252
NomeUsuario: beatrizhvieira
LocalUsuario: São José, SC
Titulo: Festa alemã de qualidade
TextoRevisao: Ao contrário de muitas outras festa desse porte pelo pais, essa vc não precisa comprar camarote para aproveitar melhor a festa. O local é enorme, se não me engano são 3 pavilhões, todos com shows.A festa é alemã, e só toca musica alemã, mas é muito divertido.Aos sábados tem entrada gratuita até um determinado horário, mas sinceramente n]ao aconselho entrar antes somente para não pagar entrada. O desfile acontece um pouco distant

IdRevisao: 666707772
NomeUsuario: barbaramD5897XQ
LocalUsuario: none
Titulo: Experiência única
TextoRevisao: Festa especial! De dia para famílias, de noite para jovens e adultos. Muita animação, muita organização, muita cerveja! Cervejas com preço super justo, variedade espetacular. Amei e gostaria de voltar sempre.
DataExperiencia: Data da experiência: outubro de 2018
Pontos: 5.0
IdRevisao: 666620180
NomeUsuario: Luysylveryo
LocalUsuario: Blumenau, SC
Titulo: OKTOBERFEST para a família
TextoRevisao: A festa vem a cada ano evoluindo e novamente se tornando uma festa para as famílias conhecerem e se divertirem com a cultura da região!
Nota 10.
DataExperiencia: Data da experiência: outubro de 2018
Pontos: 5.0
IdRevisao: 661667971
NomeUsuario: Mauerberg
LocalUsuario: Nova Odessa, SP
Titulo: É como estar na Octoberfest de Munique.
TextoRevisao: Uma grande festa do folclore alemão , bom para que toma suco de uva , ótimo para quem toma chopp.Uma festa para tarde e noite
DataExperiencia: Data

IdRevisao: 644134260
NomeUsuario: welsonr175
LocalUsuario: none
Titulo: Cerveja e diversão!
TextoRevisao: Festa bem estruturada e com muita descontração. Comida boa e muita cerveja , inclusive rótulos artesanais muito interessantes, dão um toque especial ao evento. Recomendado!
DataExperiencia: Data da experiência: janeiro de 2019
Pontos: 4.0
IdRevisao: 643866901
NomeUsuario: Vinícius R
LocalUsuario: Aracaju, SE
Titulo: Diversão e cerveja: ciente de pagar por isso, é só festa!
TextoRevisao: Não bebo. Mas fui à Oktoberfest (hehe). Numa viagem em família com tios que bebem, a experiência da festa não foi de todo ruim. Como sempre fui curioso por outras culturas, a Oktoberfest traz justamente isso: não é um evento tal qual um Rock in Rio, uma festa nordestina ou um festival de sertanejo, que abrange múltiplas culturas e pessoas.Não, a Oktoberfest é sim (e faz jus) uma festa tipicamente alemã. E o público que frequenta a festa, mesmo turista, traz algo consigo de alemão. Foi uma das primei

IdRevisao: 631395885
NomeUsuario: ZacariasPires
LocalUsuario: São José do Rio Preto, SP
Titulo: Depois de Munique na Alemanha esta é a melhor festa da cerveja no mundo!
TextoRevisao: Não dá para falar tudo que é de bom esta festa. Só indo mesmo para ver o quanto é bonita, alegre, festiva. Um espetáculo de pouco mais de 15 dias que valem a pena ser vividos. Tudo de bom.
DataExperiencia: Data da experiência: outubro de 2018
Pontos: 5.0
IdRevisao: 631359225
NomeUsuario: malusilvajesus
LocalUsuario: Blumenau, SC
Titulo: Melhor Festa de Outubro
TextoRevisao: A oktoberfest é sem dúvidas a melhor festa alemã do Brasil....melhor chopp....melhor comida típica...dança e animação.
DataExperiencia: Data da experiência: outubro de 2018
Pontos: 5.0
IdRevisao: 631159712
NomeUsuario: rodrigoalmrmeidaas
LocalUsuario: none
Titulo: Festão!
TextoRevisao: Fomos na segunda e terça-feira da segunda semana da festa de 2018. Estávamos com um pouco de receio de ir no início da semana, mas valeu muito a pena.Nes

IdRevisao: 627544132
NomeUsuario: fdsbastos
LocalUsuario: Rio de Janeiro, RJ
Titulo: Só vai, vale a pena!
TextoRevisao: Muito divertido, tudo limpo e organizado como eu nunca vi. Preços justos, dentro e fora dos pavilhões, muitas opções de comida e bebida, shows rolando o tempo todo. Muito seguro para ir com família e amigos, recomendado!
DataExperiencia: Data da experiência: outubro de 2018
Pontos: 5.0
IdRevisao: 627532940
NomeUsuario: Aryane_Spadotto
LocalUsuario: Canoinhas, SC
Titulo: Diversão
TextoRevisao: Fui à festa no dia 13/10/18. A limpeza é feita constantemente e estava muito bem organizada. Haviam muitos seguranças e policiais. Pessoas animadas, chope e comida boa.
DataExperiencia: Data da experiência: outubro de 2018
Pontos: 5.0
IdRevisao: 627382333
NomeUsuario: 837robsonb
LocalUsuario: none
Titulo: É possivel ir com crianças!!!
TextoRevisao: O local é muito grande. Os setores abrem as 11h já com muita coisa aberta na vila. Logo que vc entra, os monitores colocam identifica

...

IdRevisao: 426622782
NomeUsuario: Silvina B
LocalUsuario: Belem, Brazil
Titulo: Tradição 
TextoRevisao: Uma festa, sem duvida, para manter a tradição dos colonizadores, especialmente alemães. Há apresentação de danças folclóricas e bandas de música no mesmo estilo. Para quem não bebe cerveja, aproveite a culinária alemã, tem bastante variedade. Compre um souvenir, aprecie os adultos e crianças da cidade vestidos com trajes típicos. Um passeio que vale a pena e, imperdível, para quem ama cerveja.
DataExperiencia: Data da experiência: outubro de 2016
Pontos: 4.0
IdRevisao: 426513341
NomeUsuario: Rodrigo C
LocalUsuario: none
Titulo: Para todos os gostos
TextoRevisao: Pessoal de todas as idades, estado civil ou companhia, independente dos interesses, sairá da Oktoberfest bastante satisfeito. Tem opções para a família, para casais, para os solteiros, etc. Basta escolher o melhor dia e horário para visitar a Vila Germânica.
DataExperiencia: Data da experiência: outubro de 2016
Pontos: 5.0
Id

IdRevisao: 397031442
NomeUsuario: IracemaR3
LocalUsuario: none
Titulo: festa alemã no brasil
TextoRevisao: Festa alegre e maravilhosa, encontro de todos os povos brasileiros para assistir as tradições de dança, comidas e chopes alemãs, apresentadas por povo catarinense. Adorei.
DataExperiencia: Data da experiência: outubro de 2015
Pontos: 5.0
IdRevisao: 395160667
NomeUsuario: Vanessa M
LocalUsuario: Blumenau, SC
Titulo: Dá-lhe chopp
TextoRevisao: A festa traz um pouco da gastronomia alemã, algumas bandas da Alemanha são contratadas e muito chopp! Se você está com crianças vá durante o dia ou durante a semana a noite, pois são dias mais tranquilos. Sempre tem um pavilhão com Chopp artesanal que vale a pena visitar. Mas se você quer beber muito e arrumar um "peguete" vá sexta e sábado à noite. DICA: reserve hotel com 10 meses de antecedência para conseguir vaga, pois 8 meses antes da festa já tem over book.
DataExperiencia: Data da experiência: outubro de 2015
Pontos: 4.0
IdRevisao: 3936

Message: element click intercepted: Element <span class="taLnk ulBlueLinks" onclick="widgetEvCall('handlers.clickExpand',event,this);">...</span> is not clickable at point (485, 574). Other element would receive the click: <p class="partial_entry">...</p>
  (Session info: chrome=77.0.3865.90)

IdRevisao: 370256557
NomeUsuario: girardiorlando
LocalUsuario: Rodeio, SC
Titulo: OKTOBERFEST
TextoRevisao: após as enchentes de Blumenau e Vale do Itajaí e para voltar a criar ânimo a nossa população, pensou-se em criar algo de festivo para as famílias de Blumenau e região. Surgiu em 1984 a OKTOBERFEST, hoje a segunda maior festa de CHOPP do mundo...Mais
DataExperiencia: Data da experiência: outubro de 2015
Pontos: 5.0
IdRevisao: 370154660
NomeUsuario: diegor520
LocalUsuario: Blumenau, SC
Titulo: Ou ame ou odeie.
TextoRevisao: Eu ja amei, não ficava sem ir, hoje em dia detesto, a cidade fica cheia de mais, muita gente porca, sem cordialidade nem respeito. O transito já caótico fica pior ainda. A

IdRevisao: 361020270
NomeUsuario: marcelolA6861QS
LocalUsuario: Belo Horizonte, MG
Titulo: Oktoberfest, o Primeiro Mundo logo ali, a alegria, a felicidade, também.
TextoRevisao: Já estive várias vezes nesta festa. Há alguns anos realizamos um encontro de família, fomos todos os irmãos com maridos e mulheres.. Inesquecível. Você desembarca na cidade de Navegantes e viaja 60 minutos de ônibus até a linda Blumenau. Antigamente este traslado era fornecido pelas empresas aéreas, hoje é pago , custando cerca de R$50,00, sendo os horários atrelados a cada voo. Por fidelidade sempre me hospedo no IBIS .Lembro que em qualquer hotel as reservas devem ser feitas com muiiiita antecedência.A cidade é uma festa completa, desfiles, bandas nas praças , chope por todo lado, a toda hora. O ápice se dá após as 17,00 horas quando tudo acontece na chamada " Vila Germânica".Todos para lá se dirigem. São vários galpões, cada qual com uma banda típica, algumas da região e outras vindas da própria Alemanha.Aí 

IdRevisao: 335962203
NomeUsuario: shirley0212
LocalUsuario: Itajaí, SC
Titulo: oktober
TextoRevisao: Linda festa, adoroooo , com muitas atrações, é enorme o local da festa, tem muita variedade de comida alemã e bebida, tem atrações para crianças, a música alemã esta sempre presente em todos os pavilhões e o desfile da oktober na cidade é lindo.
DataExperiencia: Data da experiência: outubro de 2015
Pontos: 5.0
IdRevisao: 335810144
NomeUsuario: Flavia1972
LocalUsuario: São Paulo, SP
Titulo: Interessante
TextoRevisao: Conhecemos o local onde é realizado o Oktoberfest em Blumenau. O lugar é enorme, com uma impressionante infra estrutura , por ser dezembro, estava todo enfeitado para o Natal..A organização , limpeza e segurança são vistos em todo em todo o complexo. Gostamos das lojas de artesanatos e de alimentos. Não deixem de provar os chocolates que são deliciosos.
DataExperiencia: Data da experiência: dezembro de 2015
Pontos: 4.0
IdRevisao: 335686736
NomeUsuario: Roseane N
LocalUsuario